Install python requirements using:
```
pip install oauth2client
pip install google-api-python-client
pip install gspread
```

Replace `CREDENTIAL_FILE` with the path to the service account key file. Key file can be retrieved from:
https://drive.google.com/a/harvard.edu/file/d/0BxTcuuFIY3htdTVnd1lvYm5vWjg/view?usp=sharing


Share documents with `doc-revisions@appspot.gserviceaccount.com` (requires edit access) in order to view revisions.

In [4]:
# location of service account json credential file for doc-revisions@appspot.gserviceaccount.com
CREDENTIAL_FILE = 'doc-revisions-98c56004e106.json'

In [5]:
from oauth2client.service_account import ServiceAccountCredentials
from apiclient.discovery import build
import requests
import webbrowser
from httplib2 import Http

scope = ['https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE, scope)

def generate_revision_url(file_id, start, end):
    '''
    generate the url for downloading revision details (using undocumented google api endpoint)
    '''
    base_url = 'https://docs.google.com/document/d/{file_id}/revisions/load?id={file_id}&start={start}&end={end}'
    url = base_url.format(file_id=file_id,start=start,end=end)
    return url
    
def get_last_revision_id(file_id, credentials):
    '''
    get the id of the last revision to a document, using offical google api v2
    '''
    service = build('drive', 'v3', credentials=credentials)
    revision_metainfo = service.revisions().list(fileId=df_docs.file_id[0]).execute()
    return revision_metainfo['revisions'][-1]['id']

def download_revision_details(file_id, credentials):
    '''
    download json-like data with revision info
    '''
    http_auth = credentials.authorize(Http())
    last_revision_id = get_last_revision_id(file_id, credentials)
    url = generate_revision_url(file_id,1,last_revision_id)
    return http_auth.request(url)[1]

In [6]:
import gspread
import pandas as pd
from StringIO import StringIO

# file id for google spreadsheet with document info
# (https://docs.google.com/spreadsheets/d/1hanr8FJj4Wbmhjm6-_T8JMNe_X0LKn9Xbm0W14zJIFU)
FILE_ID_DOC_INFO = '1hanr8FJj4Wbmhjm6-_T8JMNe_X0LKn9Xbm0W14zJIFU'

def open_gspread(file_id, credentials):
    '''
    returns pandas dataframe from google spreadsheet specified by url key
    have to share google sheet with vpaldata@vpal-data.iam.gserviceaccount.com 
    '''
    scope = ['https://spreadsheets.google.com/feeds']
    credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIAL_FILE, scope)
    client = gspread.authorize(credentials)
    spreadsheet = client.open_by_key(file_id)
    csv = spreadsheet.sheet1.export(format='csv')
    return pd.read_csv(StringIO(csv))

# https://docs.google.com/spreadsheets/d/1hanr8FJj4Wbmhjm6-_T8JMNe_X0LKn9Xbm0W14zJIFU
df_docs = open_gspread(FILE_ID_DOC_INFO, credentials)

df_docs

,name,url,file_id
0,TranscriptExtraction,https://docs.google.com/document/d/1rV_W6lAF68...,1rV_W6lAF68WVNV02KdDVvRE73BUsYD1I2o9uBhSG5RI
1,Accessibility Project Summary,https://docs.google.com/document/d/1ptVNmlOuc9...,1ptVNmlOuc9r4iOJJyyM1qM68P72kpBquUBybyN5pEss
2,InternalMemo_LOLx,https://docs.google.com/document/d/1nItkGu-LLJ...,1nItkGu-LLJD5rFm8BaQpz5CUQdEl7wcJK5MssrZoX44
3,HarvardX_Assessment,https://docs.google.com/document/d/1wrb7u4Kzf1...,1wrb7u4Kzf1Yl7Qn4WEd9Hn6MSl9oU0UT5Sau-LgXp9g
4,Xoces,https://docs.google.com/document/d/1Fg6PEkluhN...,1Fg6PEkluhNSRJeXunZ7jZS0E1i0ld5Ep7A1GAx9VLBo
5,Spark grant,https://docs.google.com/document/d/1aSAA-ZA8bG...,1aSAA-ZA8bGvJSpgFhVgJu89EDMPMRm5IrcdeV85JmJE


In [7]:
download_revision_details(df_docs.file_id[4], credentials)

')]}\'\n{"changelog":[[{"ty":"mlti","mts":[{"ty":"as","st":"paragraph","sm":{"ps_al_i":true,"ps_awao_i":true,"ps_ifl_i":true,"ps_il_i":true,"ps_ir_i":true,"ps_klt_i":true,"ps_kwn_i":true,"ps_ls_i":true,"ps_sa_i":true,"ps_sb_i":true,"ps_sm_i":true},"ei":1,"si":1,"fm":false},{"ty":"as","st":"language","sm":{"lgs_l":"en"},"ei":0,"si":0,"fm":false},{"ty":"as","st":"text","sm":{"ts_bd_i":true,"ts_bgc_i":true,"ts_ff_i":true,"ts_fgc_i":true,"ts_fs_i":true,"ts_it_i":true,"ts_sc_i":true,"ts_st_i":true,"ts_un_i":true,"ts_va_i":true},"ei":1,"si":0,"fm":false},{"ty":"as","st":"headings","sm":{"hs_h1":{"sdef_ps":{"ps_al":0,"ps_al_i":true,"ps_awao":false,"ps_awao_i":true,"ps_hd":0,"ps_hdid":"","ps_ifl":0.0,"ps_ifl_i":true,"ps_il":0.0,"ps_il_i":true,"ps_ir":0.0,"ps_ir_i":true,"ps_klt":true,"ps_klt_i":false,"ps_kwn":true,"ps_kwn_i":false,"ps_ls":1.15,"ps_ls_i":true,"ps_ltr":true,"ps_sa":6.0,"ps_sa_i":false,"ps_sb":20.0,"ps_sb_i":false,"ps_shd":false,"ps_sm":0,"ps_sm_i":true,"ps_ts":{"cv":{"op":"set","